In [1]:
import argparse
import base64
import json
import cv2

import numpy as np
import socketio
import eventlet
import eventlet.wsgi
import time
from PIL import Image
from PIL import ImageOps
from flask import Flask, render_template
from io import BytesIO

from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
import tensorflow as tf
#tf.python.control_flow_ops = tf

def preprocess(image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # get shape and chop off 1/3 from the top
    #shape = image.shape
    # note: numpy arrays are (row, col)!
    #image = image[shape[0]-70:shape[0]-25, 0:shape[1]]
    #image = cv2.resize(image, (200,66), interpolation=cv2.INTER_AREA)
    #image = cv2.resize(image, (64,64), interpolation=cv2.INTER_AREA)
    return image



sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None





class SimplePIController:
    def __init__(self, Kp, Ki):
        self.Kp = Kp
        self.Ki = Ki
        self.set_point = 0.
        self.error = 0.
        self.integral = 0.

    def set_desired(self, desired):
        self.set_point = desired

    def update(self, measurement):
        # proportional error
        self.error = self.set_point - measurement

        # integral error
        self.integral += self.error
        
        cal_throttle = self.Kp * self.error + self.Ki * self.integral
        throttle = min(cal_throttle, 1)
        
        return throttle


controller = SimplePIController(0.1, 0.002)
set_speed = 12
controller.set_desired(set_speed)



@sio.on('telemetry')
def telemetry(sid, data):
    # The current steering angle of the car
    steering_angle = data["steering_angle"]
    # The current throttle of the car
    throttle = data["throttle"]
    # The current speed of the car
    speed = data["speed"]
    # The current image from the center camera of the car
    imgString = data["image"]
    image = Image.open(BytesIO(base64.b64decode(imgString)))
    image_pre = np.asarray(image)
    image_array = preprocess(image_pre)
    #image_array = np.asarray(image)
    transformed_image_array = image_array[None, :, :, :]
    # This model currently assumes that the features of the model are just the images. Feel free to change this.
    steering_angle = 1.0*float(model.predict(transformed_image_array, batch_size=1))
    # The driving model currently just outputs a constant throttle. Feel free to edit this.
    throttle = controller.update(float(speed))
#     if float(speed) > 20:
#         throttle = 0.1
#     else:
#         throttle = 0.2
    print(steering_angle, throttle)
    send_control(steering_angle, throttle)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit("steer", data={
    'steering_angle': steering_angle.__str__(),
    'throttle': throttle.__str__()
    }, skip_sid=True)


if __name__ == '__main__':
#     parser = argparse.ArgumentParser(description='Remote Driving')
#     parser.add_argument('model', type=str,
#     help='Path to model definition json. Model weights should be on the same path.')
#     args = parser.parse_args()
#     with open(args.model, 'r') as jfile:
#         model = model_from_json(jfile.read())
    with open('model.json', 'r') as jfile:
        model = model_from_json(jfile.read())

    model.compile("adam", "mse")
    weights_file = 'model.h5'
    model.load_weights(weights_file)

    # wrap Flask application with engineio's middleware
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

Using TensorFlow backend.
(4703) wsgi starting up on http://0.0.0.0:4567
(4703) accepted ('127.0.0.1', 42513)


connect  b87462b31c4b40dcb8b7c9d7af242bd7
-0.14934681355953217 1
-0.14934681355953217 1
-0.1499146968126297 1
-0.14802595973014832 1
-0.14802595973014832 1
-0.14802595973014832 1
-0.14834924042224884 1
-0.14834924042224884 1
-0.14834924042224884 1
-0.13367535173892975 0.8571006000000001
-0.13367535173892975 0.8700614000000001
-0.13367535173892975 0.8830222000000001
-0.13504114747047424 0.7139946
-0.13504114747047424 0.723387
-0.13504114747047424 0.7327794000000001
-0.1357337236404419 0.5435778
-0.1357337236404419 0.5490762
-0.1357337236404419 0.5545746
-0.16113722324371338 0.4343988
-0.16113722324371338 0.43743299999999996
-0.16113722324371338 0.44046719999999995
-0.16303658485412598 0.2919294000000001
-0.16303658485412598 0.29199160000000013
-0.16303658485412598 0.29205380000000014
-0.10805864632129669 0.23543460000000016
-0.10805864632129669 0.23438540000000013
-0.10805864632129669 0.23333620000000016
-0.11809670180082321 0.1943838000000001
-0.11809670180082321 0.19259140000000013
-0

0.2442764788866043 0.05726459999999994
0.2442764788866043 0.05202859999999998
0.2442764788866043 0.04679259999999996
0.3207986056804657 0.08720159999999993
0.3207986056804657 0.0828605999999999
0.3207986056804657 0.07851959999999991
0.24690128862857819 0.06519240000000004
0.24690128862857819 0.06067520000000004
0.24690128862857819 0.05615800000000004
0.19425608217716217 0.06091259999999998
0.19425608217716217 0.056577199999999994
0.19425608217716217 0.052241800000000005
0.14377090334892273 0.09537720000000005
0.14377090334892273 0.09197260000000002
0.14377090334892273 0.08856800000000004
0.08904626965522766 0.10104479999999996
0.08904626965522766 0.09795159999999994
0.08904626965522766 0.09485839999999998
0.11933258175849915 0.10040459999999995
0.11933258175849915 0.09748079999999995
0.11933258175849915 0.09455699999999995
0.26118695735931396 0.09448920000000002
0.26118695735931396 0.09162140000000002
0.26118695735931396 0.08875360000000004
0.429881751537323 0.09376019999999999
0.42988

0.18824626505374908 0.4523123999999999
0.18824626505374908 0.45460499999999987
0.18824626505374908 0.45689759999999985
0.17609921097755432 0.4546409999999998
0.17609921097755432 0.45684439999999976
0.17609921097755432 0.45904779999999973
0.08894018083810806 0.5678207999999998
0.08894018083810806 0.5721137999999998
0.08894018083810806 0.5764067999999998
0.05311167240142822 0.5192039999999998
0.05311167240142822 0.5222911999999998
0.05311167240142822 0.5253783999999998
0.01580054871737957 0.46134959999999986
0.01580054871737957 0.4631207999999999
0.01580054871737957 0.4648919999999999
0.005768054164946079 0.5463252
0.005768054164946079 0.5496584
0.005768054164946079 0.5529915999999999
0.04144172742962837 0.5704007999999998
0.04144172742962837 0.5740099999999999
0.04144172742962837 0.5776191999999998
-0.00806538388133049 0.6046373999999999
-0.00806538388133049 0.6087055999999998
-0.00806538388133049 0.6127737999999998
-0.03817523270845413 0.4533155999999997
-0.03817523270845413 0.45417739

message handler error
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/engineio/server.py", line 398, in _trigger_event
    return self.handlers[event](*args)
  File "/usr/local/lib/python3.5/dist-packages/socketio/server.py", line 520, in _handle_eio_message
    self._handle_event(sid, pkt.namespace, pkt.id, pkt.data)
  File "/usr/local/lib/python3.5/dist-packages/socketio/server.py", line 456, in _handle_event
    self._handle_event_internal(self, sid, data, namespace, id)
  File "/usr/local/lib/python3.5/dist-packages/socketio/server.py", line 459, in _handle_event_internal
    r = server._trigger_event(data[0], namespace, sid, *data[1:])
  File "/usr/local/lib/python3.5/dist-packages/socketio/server.py", line 488, in _trigger_event
    return self.handlers[namespace][event](*args)
  File "<ipython-input-1-a1015841957a>", line 75, in telemetry
    steering_angle = data["steering_angle"]
TypeError: 'NoneType' object is not subscriptable
message handle

...

-0.03982621431350708 0.0953707999999997
-0.03982621431350708 0.09186219999999967
-0.03982621431350708 0.08835359999999964
-0.01909191720187664 0.09580999999999965
-0.01909191720187664 0.09251639999999964
-0.01909191720187664 0.08922279999999963
-0.018276317045092583 0.09400759999999964
-0.018276317045092583 0.09087239999999963
-0.018276317045092583 0.08773719999999965
0.03929159790277481 0.09075259999999963
0.03929159790277481 0.08773799999999962
0.03929159790277481 0.08472339999999962
-0.002577277598902583 0.09573379999999976
-0.002577277598902583 0.09299419999999975
-0.002577277598902583 0.09025459999999977
-0.021892612800002098 0.09047299999999975
-0.021892612800002098 0.08779139999999974
-0.021892612800002098 0.08510979999999974
0.03396132215857506 0.09816679999999967
0.03396132215857506 0.09579379999999968
0.03396132215857506 0.09342079999999967
-0.025808943435549736 0.10262479999999967
-0.025808943435549736 0.10047879999999966
-0.025808943435549736 0.09833279999999965
-0.02911495

127.0.0.1 - - [25/Oct/2017 18:26:25] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 110.768760


-0.06064147502183914 0.10156459999999984
-0.06064147502183914 0.10142059999999983
-0.06064147502183914 0.10127659999999983
-0.0816139429807663 0.10628359999999977
-0.0816139429807663 0.10624059999999977
-0.0816139429807663 0.10619759999999977
-0.10140099376440048 0.10713379999999965
-0.10140099376440048 0.10710999999999966
-0.10140099376440048 0.10708619999999967
-0.0951579287648201 0.09960619999999965
-0.0951579287648201 0.09943619999999964
-0.0951579287648201 0.09926619999999964
-0.1097363606095314 0.1000753999999997
-0.1097363606095314 0.0999245999999997
-0.1097363606095314 0.0997737999999997
0.022732729092240334 0.10834399999999968
0.022732729092240334 0.10836419999999967
0.022732729092240334 0.10838439999999969
0.007201247848570347 0.10790479999999976
0.007201247848570347 0.10791519999999977
0.007201247848570347 0.10792559999999977
0.026727864518761635 0.09866419999999981
0.026727864518761635 0.0984927999999998
0.026727864518761635 0.0983213999999998
0.023941488936543465 0.1001593

0.002494626911357045 0.11341459999999969
0.002494626911357045 0.11361259999999969
0.002494626911357045 0.11381059999999969
0.028624163940548897 0.10524679999999964
0.028624163940548897 0.10527299999999964
0.028624163940548897 0.10529919999999965
-0.008066541515290737 0.1078345999999996
-0.008066541515290737 0.1079099999999996
-0.008066541515290737 0.1079853999999996
-0.0427376814186573 0.10425619999999977
-0.0427376814186573 0.10425699999999977
-0.0427376814186573 0.10425779999999976
0.007031300105154514 0.09867919999999973
0.007031300105154514 0.09857059999999973
0.007031300105154514 0.09846199999999973
-0.02077369950711727 0.10340239999999976
-0.02077369950711727 0.10339279999999974
-0.02077369950711727 0.10338319999999974
-0.11223159730434418 0.10564819999999969
-0.11223159730434418 0.10568319999999969
-0.11223159730434418 0.10571819999999969
-0.10086795687675476 0.09941899999999959
-0.10086795687675476 0.09932979999999958
-0.10086795687675476 0.09924059999999958
-0.0310719031840562

127.0.0.1 - - [25/Oct/2017 18:26:58] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 83.634583
127.0.0.1 - - [25/Oct/2017 18:26:58] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 55.135255
wsgi exiting
(4703) wsgi exited, is_accepting=True
